# CNN Facial Recognition

# 1. Introduction
Hey guys, the Stats Whisper is back with another task at hand. This time we will be using a convulution neural network to resolve a doopleganger debate. 





## CNN Explianed

- images
- videos 
- mathematically explained


# 2. Application

## 2.1 Creating Basic CNN using Local Data with 3 classes
    -Neymar
    -Rafinha
    -Cristiano Ronaldo

In [1]:
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras.callbacks import TensorBoard
K.set_image_dim_ordering('th')
from time import time

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(3, 250, 250)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(3))
model.add(Activation('sigmoid'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

Using TensorFlow backend.


In [2]:
batch_size = 5 

train_datagen = ImageDataGenerator()

test_datagen = ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(
        'data/train',  # this is the target directory
        target_size=(250, 250),  # all images will be resized to 250x250
        batch_size=batch_size,
        class_mode='categorical')  # since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        'data/validation',
        target_size=(250, 250),
        batch_size=batch_size,
        class_mode='categorical')

tensorboard = TensorBoard(log_dir="tensorBoard/".format(time()), write_images=True, write_graph=True)



Found 30 images belonging to 3 classes.
Found 30 images belonging to 3 classes.


In [3]:
model.fit_generator(
        train_generator,
        steps_per_epoch=20,
        epochs=5,
        validation_data=validation_generator,
        validation_steps=10,
        callbacks = [tensorboard])
model.save_weights('basic3_cnn.h5')  # always save your weights after training or during training

Epoch 1/5


/Users/juliogonzalez/Library/Python/3.6/lib/python/site-packages/PIL/Image.py:914: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


20/20 [==============================] - 47s 2s/step - loss: 8.6429 - acc: 0.2600 - val_loss: 10.3156 - val_acc: 0.3600
Epoch 2/5
12/20 [=================>............] - ETA: 13s - loss: 8.5858 - acc: 0.3167

KeyboardInterrupt: 

# 3. Advanced Aplication

## 3.1 ResNet50 w/same local data
-Emphasize significance of pre-trained weights

In [1]:
#see if you can run it locally first, if not then will need to use Google's colab

from keras.engine import  Model
from keras_vggface.vggface import VGGFace
from keras.models import Sequential,Input,Model
from keras import layers
from keras.layers import Dense, Flatten,Input,Dropout,Activation,Conv2D
import keras

#custom parameters
nb_class = 2
hidden_dim = 512

resnet50_model = VGGFace(model='resnet50',include_top=False, input_shape=(224, 224, 3))
last_layer = resnet50_model.get_layer('avg_pool').output
x = Flatten(name='flatten')(last_layer)
out = Dense(nb_class, activation='softmax', name='classifier')(x)
custom_resnet50_model = Model(resnet50_model.input, out)


batch_size = 64
epochs = 20

custom_resnet50_model.compile(loss=keras.losses.categorical_crossentropy,
         optimizer=keras.optimizers.Adam(),metrics=['accuracy'])

Using TensorFlow backend.


In [2]:
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
from keras.callbacks import TensorBoard
K.set_image_dim_ordering('th')
from time import time
batch_size = 5 

train_datagen = ImageDataGenerator(data_format="channels_last")

test_datagen = ImageDataGenerator(data_format="channels_last")

train_generator = train_datagen.flow_from_directory(
        'data/train',  # this is the target directory
        target_size=(224, 224),  # all images will be resized to 250x250
        batch_size=batch_size,
        class_mode='categorical')  # since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        'data/test',
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='categorical')

tensorboard = TensorBoard(log_dir="tensorBoard/".format(time()), write_images=True, write_graph=True)

custom_resnet50_model.fit_generator(
        train_generator,
        steps_per_epoch=20,
        epochs=5,
        validation_data=validation_generator,
        validation_steps=10)


Found 70 images belonging to 2 classes.
Found 70 images belonging to 2 classes.
Epoch 1/5
 9/20 [============>.................] - ETA: 1:06 - loss: 2.2515 - acc: 0.7111

KeyboardInterrupt: 

## 3.2 ResNet50 on LFW

In [0]:
#Since we are going to use Google's colab we need to run the following code
#see if you can hide it if possible
!pip install keras_vggface
!pip install Keras==2.2
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
from keras.engine import  Model
from keras_vggface.vggface import VGGFace
from keras.models import Sequential,Input,Model
from keras import layers
from keras.layers import Dense, Flatten,Input,Dropout,Activation,Conv2D
import keras

#custom parameters
nb_class = 2
hidden_dim = 512

resnet50_model = VGGFace(model='resnet50',include_top=False, input_shape=(224, 224, 3))
last_layer = resnet50_model.get_layer('avg_pool').output
x = Flatten(name='flatten')(last_layer)
out = Dense(nb_class, activation='softmax', name='classifier')(x)
custom_resnet50_model = Model(resnet50_model.input, out)


batch_size = 64
epochs = 20

custom_resnet50_model.compile(loss=keras.losses.categorical_crossentropy,
         optimizer=keras.optimizers.Adam(),metrics=['accuracy'])

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
from keras.callbacks import TensorBoard
K.set_image_dim_ordering('th')
from time import time
batch_size = 5 

train_datagen = ImageDataGenerator(data_format="channels_last")

test_datagen = ImageDataGenerator(data_format="channels_last")

train_generator = train_datagen.flow_from_directory(
        '/content/gdrive/My Drive/data/train',  # this is the target directory
        target_size=(224, 224),  # all images will be resized to 250x250
        batch_size=batch_size,
        class_mode='categorical')  # since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        '/content/gdrive/My Drive/data/test',
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='categorical')

tensorboard = TensorBoard(log_dir="tensorBoard/".format(time()), write_images=True, write_graph=True)

custom_resnet50_model.fit_generator(
        train_generator,
        steps_per_epoch=10,
        epochs=5,
        validation_data=validation_generator,
        validation_steps=10)